# Cantera Tutorial: Python

## Getting Started

Let's get started with Cantera by importing the Cantera and NumPy libraries. We can also print the version of Cantera that we're using with the `__version__` attribute from the `cantera` module, typically aliased as `ct`.

In [1]:
import cantera as ct
import numpy as np

print(f"Using Cantera version {ct.__version__}")

Using Cantera version 2.4.0


When using Cantera, the first thing you usually need is an object representing some phase of matter. Here, we'll create a gas mixture:

In [2]:
gas1 = ct.Solution('gri30.cti')

To view the state of the mixture, *call* the `gas1` object as if it were a function:

In [3]:
gas1()


  gri30:

       temperature             300  K
          pressure          101325  Pa
           density       0.0818891  kg/m^3
  mean mol. weight         2.01588  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy           26470        5.336e+04     J
   internal energy     -1.2109e+06       -2.441e+06     J
           entropy           64914        1.309e+05     J/K
    Gibbs function     -1.9448e+07        -3.92e+07     J
 heat capacity c_p           14312        2.885e+04     J/K
 heat capacity c_v           10187        2.054e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2              1                1         -15.7173
     [  +52 minor]              0                0



What you have just done is created an object `gas1` that implements GRI-Mech 3.0, the 53-species, 325-reaction natural gas combustion mechanism developed by Gregory P. Smith, David M. Golden, Michael Frenklach, Nigel W. Moriarty, Boris Eiteneer, Mikhail Goldenberg, C. Thomas Bowman, Ronald K. Hanson, Soonho Song, William C. Gardiner, Jr., Vitali V. Lissianski, and Zhiwei Qin.  See the [GRI-Mech Home Page](http://combustion.berkeley.edu/gri-mech/) for more information.

The `gas1` object has properties you would expect for a gas mixture: a temperature, a pressure, species mole and mass fractions, etc. As we will soon see, it has many more properties.

The summary of the state of `gas1` that you found above shows that the new objects created from the `gri30.cti` input file start out with a temperature of 300 K, a pressure of 1 atm, and have a composition that consists of only one species, in this case hydrogen. There is nothing special about H2—it just happens to be the first species listed in the input file defining GRI-Mech 3.0. In general, whichever species is listed first will initially have a mole fraction of 1.0, and all others will be zero.

## Setting the State

The state of the object can easily be changed. For example:

In [4]:
gas1.TP = 1200, 101325

sets the temperature to 1200 K and the pressure to 101325 Pa (Cantera always uses SI units + kmol). After this statement, calling `gas1()` results in:

In [5]:
gas1()


  gri30:

       temperature            1200  K
          pressure          101325  Pa
           density       0.0204723  kg/m^3
  mean mol. weight         2.01588  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      1.3296e+07         2.68e+07     J
   internal energy      8.3462e+06        1.682e+07     J
           entropy           85228        1.718e+05     J/K
    Gibbs function     -8.8978e+07       -1.794e+08     J
 heat capacity c_p           15378          3.1e+04     J/K
 heat capacity c_v           11253        2.269e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2              1                1         -17.9775
     [  +52 minor]              0                0



Thermodynamics generally requires that *two* properties in addition to composition information be specified to fix the intensive state of a substance (or mixture). The state of the mixture can be set using several combinations of two properties. The following are all equivalent:

In [6]:
gas1.TP = 1200, 101325            # temperature, pressure
gas1.TD = 1200, 0.0204723         # temperature, density
gas1.HP = 1.32956e7, 101325       # specific enthalpy, pressure
gas1.UV = 8.34619e6, 1/0.0204723  # specific internal energy, specific volume
gas1.SP = 85227.6, 101325         # specific entropy, pressure
gas1.SV = 85227.6, 1/0.0204723    # specific entropy, specific volume

Cantera can set and get properties on a molar basis (J/kmol) or a mass basis (J/kg). Note that the mass basis is set by default, so all the values in the previous cell are per unit mass. The basis of a `Solution` instance can be changed by assigning to the `basis` attribute of the instance:

In [7]:
gas1.basis = 'molar'
gas1.basis = 'mass'

Properties may be also **read** independently, such as

In [8]:
gas1.T

1200.0044548350836

or

In [9]:
gas1.h

13295636.190310445

or together:

In [10]:
gas1.UV

(8346238.627182945, 48.84649013545132)

In [20]:
water = ct.PureFluid('liquidvapor.cti', 'water')
water.TP = 1200, 101325
water.TPX = None, None, 1.0

AttributeError: attribute 'TPX' of 'cantera._cantera.PureFluid' objects is not writable

The composition can be set in terms of either mole fractions (`X`) or mass fractions (`Y`) by assigning to the corresponding attribute of the `Solution` instance. There are three main options to set the composition of a mixture:

* A string specifying the species names and relative mole numbers

      "CH4:1, O2:2, N2:7.52"
      
* A Python dictionary where the keys are species names and the values are relative mole numbers

      {"CH4": 1, "O2": 2, "N2": 7.52}

* A NumPy array of length `n_species`

In any of these case, the mole numbers are normalized so the sum is 1.0.

In [12]:
gas1.X = "CH4:0.8, O2:2, N2:7.52"
print(gas1.mole_fraction_dict())

{'CH4': 0.07751937984496125, 'N2': 0.7286821705426356, 'O2': 0.1937984496124031}


In [13]:
phi = 0.8
gas1.X = {'CH4':1, 'O2':2/phi, 'N2': 2*3.76/phi}
print(gas1.mole_fraction_dict())

{'CH4': 0.07751937984496125, 'N2': 0.7286821705426356, 'O2': 0.19379844961240314}


In [15]:
nsp = gas1.n_species
gas1.Y = np.ones(nsp)
gas1()


  gri30:

       temperature            1200  K
          pressure         13431.2  Pa
           density       0.0204723  kg/m^3
  mean mol. weight         15.2079  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      1.2527e+07        1.905e+08     J
   internal energy      1.1871e+07        1.805e+08     J
           entropy           17371        2.642e+05     J/K
    Gibbs function     -8.3188e+06       -1.265e+08     J
 heat capacity c_p          2898.4        4.408e+04     J/K
 heat capacity c_v          2351.7        3.576e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2        0.14234        0.0188679         -21.9478
                 H       0.284681        0.0188679          3.17231
                 O      0.0179345        0.0188679         -2.08338
                O2     0.0

One additional method is available to set the equivalence ratio directly, called [`set_equivalence_ratio()`](https://cantera.org/documentation/docs-2.4/sphinx/html/cython/thermo.html#cantera.ThermoPhase.set_equivalence_ratio). In this case, it is assumed that all C atoms are oxidized to CO2, H atoms to H2O, and S to SO2. Other atoms are assumed not to react (e.g., N ends up as N2). The signature for this method is:

    set_equivalence_ratio(phi, fuel, oxidizer)
    
where the `phi` argument is a number that represents the desired equivalence ratio of the mixture and the `fuel` and `oxidizer` represent the fuel and oxidizer mixtures in any of the formats shown before on a molar basis. For instance, to set the equivalence ratio to 0.8 with an equimolar fuel mixture of methane and propane and an oxidizer of air, the code is:

In [16]:
gas1.set_equivalence_ratio(phi, {"CH4": 1, "C3H8": 1}, "O2:1, N2:3.76")
print(gas1.mole_fraction_dict())

{'C3H8': 0.022909507445589925, 'CH4': 0.022909507445589925, 'N2': 0.7537227949599085, 'O2': 0.20045819014891184}


When the composition alone is changed, the **temperature** and **density** are held constant. This means that the pressure and other intensive properties will change. The composition can also be set in conjunction with the intensive properties of the mixture:

In [17]:
gas1.TPX = 1200, 101325, "CH4:1, O2:2, N2:7.52"
gas1()


  gri30:

       temperature            1200  K
          pressure          101325  Pa
           density        0.280629  kg/m^3
  mean mol. weight         27.6332  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      8.6194e+05        2.382e+07     J
   internal energy      5.0088e+05        1.384e+07     J
           entropy          8914.3        2.463e+05     J/K
    Gibbs function     -9.8352e+06       -2.718e+08     J
 heat capacity c_p          1397.3        3.861e+04     J/K
 heat capacity c_v          1096.4         3.03e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2       0.190114         0.220149         -28.7472
               CH4       0.095057        0.0551863          -35.961
                N2       0.714829         0.724665         -25.6789
     [  +50 minor]        

When setting the state, you can control what properties are held constant by passing the special value `None` to the property setter. For example, to change the specific volume to 2.1 m<sup>3</sup>/kg while holding entropy constant:

In [18]:
gas1.SV = None, 2.1
gas1()


  gri30:

       temperature         1383.55  K
          pressure          198234  Pa
           density         0.47619  kg/m^3
  mean mol. weight         27.6332  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      1.1224e+06        3.102e+07     J
   internal energy      7.0615e+05        1.951e+07     J
           entropy          8914.3        2.463e+05     J/K
    Gibbs function     -1.1211e+07       -3.098e+08     J
 heat capacity c_p          1439.9        3.979e+04     J/K
 heat capacity c_v            1139        3.147e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2       0.190114         0.220149         -28.5135
               CH4       0.095057        0.0551863         -35.1153
                N2       0.714829         0.724665         -25.4211
     [  +50 minor]        

Or to set the mass fractions while holding temperature and pressure constant:

In [ ]:
gas1.TPY = None, None, "CH4:1.0, O2:0.5"

## Working with a Subset of Species

In [ ]:
print(gas1.species())

Many properties of a [`Solution`](https://cantera.org/documentation/docs-2.4/sphinx/html/cython/importing.html#cantera.Solution) provide values for each species present in the phase. If you want to get values only for a subset of these species, you can use Python's "slicing" syntax to select data for just the species of interest. To get the mole fractions of just the major species in `gas1`, in the order specified, you can write:

In [ ]:
Xmajor = gas1['CH4','O2','CO2','H2O','N2'].X
print(Xmajor)

If you want to use the same set of species repeatedly, you can keep a reference to the sliced phase object:

In [ ]:
major = gas1['CH4','O2','CO2','H2O','N2']
cp_major = major.partial_molar_cp
wdot_major = major.net_production_rates
print(wdot_major)

The slice object and the original object share the same internal state, so modifications to one will affect the other.

In [ ]:
gas1.TPX = 1200, 101325, "CH4:1, N2:7.52, O2:2"
print(major.net_production_rates)
print(major.X)

## Working with Mechanism Files

In the previous example, we created an object that models an ideal gas mixture with the species and reactions of GRI-Mech 3.0, using the `gri30.cti` input file included with Cantera. This is a CTI input file and is relatively easy for humans to read and write. Cantera also supports an XML-based input file format that is easy for Cantera to parse, but hard for humans to write. Several reaction mechanism files in both formats are included with Cantera, including ones that model high-temperature air, a hydrogen/oxygen reaction mechanism, and a few surface reaction mechanisms. These files are usually located in the `data` subdirectory of the Cantera installation directory, e.g., `C:\Program Files\Cantera\data` on Windows or `/usr/local/cantera/data/` on Unix/Linux/Mac OS X machines, depending on how you installed Cantera and the options you specified.

There are a number of mechanism files included with Cantera, including the `gri30.cti` example we saw earlier.

In [21]:
from pathlib import Path
p = Path(ct.__file__)
print([c.name for c in (p.parent / "data").glob("*.cti")])

['nDodecane_Reitz.cti', 'air.cti', 'ohn.cti', 'silane.cti', 'silicon_carbide.cti', 'ptcombust.cti', 'argon.cti', 'gri30_highT.cti', 'KOH.cti', 'methane_pox_on_pt.cti', 'diamond.cti', 'h2o2.cti', 'water.cti', 'silicon.cti', 'nasa.cti', 'graphite.cti', 'nasa_gas.cti', 'gri30.cti', 'nasa_condensed.cti', 'liquidvapor.cti', 'airNASA9.cti', 'gri30_ion.cti']


Cantera input files are plain text files, and can be created with any text editor. See the document *[Defining Phases](https://cantera.org/tutorials/cti/defining-phases.html)* for more information.

A Cantera input file may contain more than one phase specification, or may contain specifications of interfaces (surfaces). Here, we import definitions of two bulk phases and the interface between them from the file `diamond.cti`:

In [ ]:
gas2 = ct.Solution('diamond.cti', 'gas')
diamond = ct.Solution('diamond.cti', 'diamond')
diamond_surf = ct.Interface('diamond.cti', 'diamond_100', [gas2, diamond])

Note that the bulk (i.e., 3D or homogenous) phases that participate in the surface reactions must also be passed as arguments to [`Interface`](http://cantera.github.io/docs/sphinx/html/cython/importing.html#cantera.Interface).

### Converting CK-format files

Cantera also comes with a script to convert CHEMKIN (CK)-format input files to the CTI format. We'll cover that in the [`chemkin_conversion.ipynb`](chemkin_conversion.ipynb) Notebook.

## Getting Help

In addition to the Sphinx-generated *[Python Module Documentation](https://cantera.org/documentation/docs-2.4/sphinx/html/index.html)*, documentation of the Python classes and their methods can be accessed from within the Python interpreter as well.

Suppose you have created a Cantera object and want to know what methads are avialable for it, and get help on using the methods:

In [22]:
g = ct.Solution("gri30.cti")

To get help on the Python class that this object is an instance of, put a question mark `?` after the variable:

In [23]:
g?

For a simple list of the properties and methods of this object:

In [24]:
dir(g)

['DP',
 'DPX',
 'DPY',
 'HP',
 'HPX',
 'HPY',
 'ID',
 'P',
 'P_sat',
 'SP',
 'SPX',
 'SPY',
 'SV',
 'SVX',
 'SVY',
 'T',
 'TD',
 'TDX',
 'TDY',
 'TP',
 'TPX',
 'TPY',
 'T_sat',
 'UV',
 'UVX',
 'UVY',
 'X',
 'Y',
 '__call__',
 '__class__',
 '__copy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_check_kinetics_species_index',
 '_check_phase_index',
 '_check_reaction_index',
 '_full_states',
 '_init_cti_xml',
 '_init_parts',
 '_references',
 'activities',
 'activity_coefficients',
 'add_reaction',
 'add_species',
 'atomic_weight',
 'atomic_weights',
 'basis',
 'binary_diff_coeffs',
 'chemical_potentials',
 'concentrations',
 'cp',
 'cp_mass',
 'cp_mole',
 'cr

To get help on a specific method, e.g. the `species_index` method:

In [25]:
g.species_index?

For properties, getting the documentation is slightly trickier, as the usual method will give you help for the *result*, e.g.:

In [26]:
g.T?

provides help on Python's `float` class. To get the help for the temperature property, ask for the attribute of the class object itself:

In [27]:
g.__class__.T?

Help can also be obtained using the `help` function:

In [28]:
help(g.species_index)

Help on built-in function species_index:

species_index(...) method of cantera.composite.Solution instance
    ThermoPhase.species_index(self, species) -> int
    
    The index of species *species*, which may be specified as a string or
    an integer. In the latter case, the index is checked for validity and
    returned. If no such species is present, an exception is thrown.



## Chemical Equilibrium

To set a gas mixture to a state of chemical equilibrium, use the `equilibrate` method:

In [29]:
g = ct.Solution("gri30.cti")
g.TPX = 300.0, ct.one_atm, "CH4:0.95, O2:2, N2:7.52"
g.equilibrate("TP")
g()


  gri30:

       temperature             300  K
          pressure          101325  Pa
           density         1.12477  kg/m^3
  mean mol. weight         27.6886  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy     -2.8724e+06       -7.953e+07     J
   internal energy     -2.9624e+06       -8.203e+07     J
           entropy          7226.7        2.001e+05     J/K
    Gibbs function     -5.0404e+06       -1.396e+08     J
 heat capacity c_p          1106.5        3.064e+04     J/K
 heat capacity c_v          806.22        2.232e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2      0.0095511        0.0110379         -29.3248
               H2O       0.181471         0.118072         -121.367
               CO2      0.0907354          0.14422         -185.873
               NO2    6.06

The above statement sets the state of object `g` to the state of chemical equilibrium holding temperature and pressure fixed. Alternatively, the specific enthalpy and pressure can be held fixed:

In [30]:
g.TPX = 300.0, ct.one_atm, "CH4:0.95, O2:2, N2:7.52"
g.equilibrate("HP")
g()


  gri30:

       temperature         2188.95  K
          pressure          101325  Pa
           density        0.153491  kg/m^3
  mean mol. weight         27.5699  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy     -2.4244e+05       -6.684e+06     J
   internal energy     -9.0258e+05       -2.488e+07     J
           entropy          9804.2        2.703e+05     J/K
    Gibbs function     -2.1703e+07       -5.984e+08     J
 heat capacity c_p          1500.5        4.137e+04     J/K
 heat capacity c_v            1199        3.306e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2     0.00183219      0.000133967         -26.0724
                 H    0.000226589      8.28395e-06         -13.0362
                 O    0.000259255      0.000150451         -16.8188
                O2      0.

Other options are:
* `'UV'` for fixed specific internal energy and specific volume
* `'SV'` for fixed specific entropy and specific volume
* `'SP'` for fixed specific entropy and pressure

How can you tell if `equilibrate` has correctly found the chemical equilibrium state? One way is to verify that the net rates of progress of all reversible reactions are zero. Here is the code to do this:

In [31]:
g.TPX = 300.0, ct.one_atm, 'CH4:0.95, O2:2, N2:7.52'
g.equilibrate('HP')

In [32]:
rf = g.forward_rates_of_progress
rr = g.reverse_rates_of_progress
for i in range(g.n_reactions):
    if g.is_reversible(i) and rf[i] != 0.0:
        print(f"{i:4d}\t{(rf[i] - rr[i])/rf[i]:10.4g}")

   0	-4.756e-15
   1	-3.332e-15
   2	-3.936e-16
   3	-3.751e-15
   4	-6.626e-15
   5	-1.503e-15
   6	 5.377e-15
   7	-1.609e-15
   8	 1.267e-14
   9	-1.587e-16
  10	-4.093e-15
  11	  7.16e-15
  12	-9.032e-15
  13	 -1.72e-15
  14	-2.147e-16
  15	-5.203e-15
  16	-1.743e-15
  17	 5.791e-15
  18	 1.957e-15
  19	-5.548e-15
  20	 1.886e-15
  21	 1.421e-15
  22	 1.223e-16
  23	 9.182e-15
  24	 2.055e-15
  25	-5.119e-15
  26	 3.383e-15
  27	 -1.44e-14
  28	-3.495e-16
  29	-1.949e-15
  30	 1.452e-14
  31	 1.048e-14
  32	 3.608e-15
  33	 -3.47e-15
  34	-3.443e-15
  35	 9.966e-15
  37	 8.611e-16
  38	-4.981e-15
  39	-1.486e-15
  40	-8.614e-15
  41	-1.518e-15
  42	-1.232e-15
  43	-6.832e-15
  44	-5.306e-15
  45	-9.065e-15
  46	  -9.2e-15
  47	-1.436e-14
  48	-7.131e-15
  49	-1.415e-15
  50	-1.151e-16
  51	  2.08e-15
  52	-7.122e-15
  53	-6.697e-15
  54	-1.066e-14
  55	 4.771e-15
  56	 1.255e-15
  57	-2.753e-15
  58	-1.199e-14
  59	-7.143e-15
  60	-1.246e-14
  61	 -1.46e-14
  62	-8.636e-15
  63	   

If the magnitudes of the numbers in this list are all very small (which in this case they are), then each reversible reaction is very nearly equilibrated, which only occurs if the gas is in chemical equilibrium.

You might be wondering how `equilibrate` works. (Then again, you might not.) Method `equilibrate` invokes Cantera's chemical equilibrium solver, which uses an element potential method. The element potential method is one of a class of equivalent *nonstoichiometric* methods that all have the characteristic that the probelm reduces to solving a set of $M$ nonlinear algebraic equations, where $M$ is the number of elements (not species). The so-called *stoichiometric* methods, on the other hand (including the Gibbs minimization), require solving $K$ nonlinear equations, where $K$ is the number of species (usually $K >> M$). See Smith and Missen's "Chemical Reaction Equilibrium Analysis" for more information on the various algorithms and their characteristics.

Cantera uses a damped Newton method to solve these equations, and does a few other things to generate a good starting guess and to produce a reasonably robust algorithm. If you want to know more about the details, look at the on-line documentated source code of Cantera C++ class [`ChemEquil.h`](https://cantera.org/documentation/docs-2.4/doxygen/html/d4/dd4/ChemEquil_8h.html).